# Plot Enemy Sprites

Run `iridisalpha.prg` with the following trace:

```
log on                                            
logname "IridisAlphaTitleEnemySpriteMovements.txt"
tr exec 6b01                                      
command 1 "mem d000 d010; mem 07f8 07ff"          
```

In [53]:
colors = {
"00":"c64_black",  
"01":"c64_white",  
"02":"c64_red",    
"03":"c64_cyan",   
"04":"c64_purple", 
"05":"c64_green",  
"06":"c64_blue",   
"07":"c64_yellow", 
"08":"c64_orange", 
"09":"c64_brown",  
"0a":"c64_ltred",  
"0b":"c64_gray1",  
"0c":"c64_lightgray",  
"0d":"c64_ltgreen",
"0e":"c64_ltblue", 
"0f":"c64_gray3",  
}
c64_to_rgb = {
"c64_black": "#000000",     
"c64_white": "#ffffff",     
"c64_red": "#880000",       
"c64_cyan":  "#aaffee",     
"c64_purple": "#cc44cc",    
"c64_green": "#00cc55",     
"c64_blue":  "#0000aa",     
"c64_yellow":  "#eeee77",   
"c64_orange":  "#dd8855",   
"c64_brown": "#664400",     
"c64_ltred": "#ff7777",  
"c64_gray1":  "#333333",    
"c64_lightgray": "#bbbbbb", 
"c64_ltgreen": "#aaff66",
"c64_ltblue":  "#0088ff",
"c64_gray3": "#bbbbbb",                             
}
hex_to_rgb = {
"00": "#000000",     
"01": "#ffffff",     
"02": "#880000",       
"03":  "#aaffee",     
"04": "#cc44cc",    
"05": "#00cc55",     
"06":  "#0000aa",     
"07":  "#eeee77",   
"08":  "#dd8855",   
"09": "#664400",     
"0a": "#ff7777",  
"0b":  "#333333",    
"0c": "#bbbbbb", 
"0d": "#aaff66",
"0e":  "#0088ff",
"0f": "#bbbbbb",                             
}

color_constants = """
BLACK        = $00
WHITE        = $01
RED          = $02
CYAN         = $03
PURPLE       = $04
GREEN        = $05
BLUE         = $06
YELLOW       = $07
ORANGE       = $08
BROWN        = $09
LTRED        = $0A
GRAY1        = $0B
GRAY2        = $0C
LTGREEN      = $0D
LTBLUE       = $0E
GRAY3        = $0F
""".split('\n')[1:-1]
color_constants = {c.split('=')[0].strip():hex_to_rgb[c[-2:].lower()] for c in color_constants}
color_constants


{'BLACK': '#000000',
 'WHITE': '#ffffff',
 'RED': '#880000',
 'CYAN': '#aaffee',
 'PURPLE': '#cc44cc',
 'GREEN': '#00cc55',
 'BLUE': '#0000aa',
 'YELLOW': '#eeee77',
 'ORANGE': '#dd8855',
 'BROWN': '#664400',
 'LTRED': '#ff7777',
 'GRAY1': '#333333',
 'GRAY2': '#bbbbbb',
 'LTGREEN': '#aaff66',
 'LTBLUE': '#0088ff',
 'GRAY3': '#bbbbbb'}

We need this to identify if the sprite is appropriate for the level. This works around the bug where sprites from a previous game can appear in the current level.

In [10]:
log_file = "DNAXPosInterrupts.txt"
input_file = open(log_file,'r')

xpos_history = []
lines = input_file.readlines()
for i in range(5, len(lines), 70):
    raw_ram = [l[9:60].split() for l in lines[i:i+3]]
    xpos_array = [int(v,16) for l in raw_ram for v in l]
    
    phase_hex = lines[i+3][9:60].strip()
    phase_dec = int(phase_hex,16)

    xpos_history += [(xpos_array, phase_dec)]


Load in the sprite data

In [15]:
import re
sprites_files = ["../iridisalpha/src/graphics/bonusphase_sprites.asm",
                 ]
sprites_data = {}
for sprites_file in sprites_files:
    input_file = open(sprites_file,'r')
    sprite_data = []
    for l in input_file.readlines():
        if "SPRITE" in l:
            if sprite_data:
                sprites_data[sprite_name] = sprite_data
            sprite_name = l[18:20].strip().lower()
            sprite_data = []
            continue

        m = re.findall(r"[0-1]{24}",l)
        if not m:
            continue
        bits = m[0]
        sprite_line = []
        for i in range(0,23,2):
            bitpair = bits[i:i+2]
            sprite_line += [bitpair]
            sprite_line += [bitpair]
        sprite_data += [sprite_line]
    if sprite_data:
        sprites_data[sprite_name] = sprite_data


In [16]:
"""
   BIT PAIR                           DESCRIPTION                          
 ------------------------------------------------------------------------- 
     00        TRANSPARENT, SCREEN COLOR                                   
     01        SPRITE MULTI-COLOR REGISTER #0 (53285) ($D025)              
     10        SPRITE COLOR REGISTER                                       
     11        SPRITE MULTI-COLOR REGISTER #I (53286) ($D026)              
"""
black_list = [
'LAND_GILBY', 'NO_SPRITE',
'BLANK_SPRITE', 'GILBY_AIRBORNE_LEFT', 'EXPLOSION_END',
'EXPLOSION_MIDDLE','EXPLOSION_START', 'GILBY_AIRBORNE_RIGHT', 
'LASER_BULLET', 'GILBY_TAKING_OFF','SPINNING_RING','GILBY_AIRBORNE_TURNING',
'TEARDROP_EXPLOSION'
]


In [54]:
from PIL import Image, ImageColor
cols=40
rows=25
bits=8
width = cols*bits
height = rows*bits

def paintSprite(origin, pixels, sprite, colors):
    multicol0, multicol1,color = colors
    colormap = {
        "01": multicol0,
        "10": color,
        "11": multicol1,
    }
    
    x_o, y_o = origin
    if sprite not in sprites_data:
        print(sprite)
        return
    bit_array = sprites_data[sprite]
    for y, l in enumerate(bit_array):
        for x,bit in enumerate(l):
            if bit == "00":
                continue
            #color = c64_to_rgb[colors[color_ram[r][c]]]
            if (x_o+x > width-1) or (y_o+y > height - 1):
                continue
            pixel_color = ImageColor.getrgb(color_constants[colormap[bit]])
            pixels[x_o+x, y_o+y] = pixel_color



Write out the sprite movement sequences to image files

In [69]:
color_seq = """
dnaSpriteOriginalLeftColorArray            .BYTE RED,ORANGE,YELLOW,GREEN,PURPLE,BLUE,END_SENTINEL 
dnaSpriteOriginalRightColorArray             .BYTE GRAY1,GRAY2,GRAY3,WHITE,GRAY3,GRAY2,END_SENTINE
dnaSpriteColorArray1 .BYTE BLUE,RED,PURPLE,GREEN,CYAN,YELLOW,WHITE,YELLOW,CYAN,GREEN,PURPLE,RED,BLUE,END_SENTINEL       
dnaSpriteColorArray1 .BYTE BLUE,RED,PURPLE,GREEN,CYAN,YELLOW,WHITE,YELLOW,CYAN,GREEN,PURPLE,RED,BLUE,END_SENTINEL               
dnaSpriteColorArray2 .BYTE BLUE,GREEN,LTBLUE,LTGREEN,CYAN,END_SENTINEL           
dnsSpriteColorArray3 .BYTE BROWN,ORANGE,YELLOW,ORANGE,BROWN,END_SENTINEL         
dnaSpriteColorArray4 .BYTE BLACK,BLACK,BLACK,RED,BLACK,BLACK,YELLOW,END_SENTINEL 
dnaSpriteColorArray5 .BYTE WHITE,GRAY3,LTGREEN,GRAY2,BLACK,BLACK,BLACK,BLACK,BLACK,BLACK,BLACK,BLACK,BLACK,END_SENTINEL
dnaSpriteColorArray6 .BYTE BLUE,LTBLUE,GRAY1,RED,GREEN,END_SENTINEL              
""".split('\n')[1:-1]
color_seq = [c.split('.BYTE')[1] for c in color_seq]
color_seq = [c.strip().split(',')[:-1] for c in color_seq]
color_seq

[['RED', 'ORANGE', 'YELLOW', 'GREEN', 'PURPLE', 'BLUE'],
 ['GRAY1', 'GRAY2', 'GRAY3', 'WHITE', 'GRAY3', 'GRAY2'],
 ['BLUE',
  'RED',
  'PURPLE',
  'GREEN',
  'CYAN',
  'YELLOW',
  'WHITE',
  'YELLOW',
  'CYAN',
  'GREEN',
  'PURPLE',
  'RED',
  'BLUE'],
 ['BLUE',
  'RED',
  'PURPLE',
  'GREEN',
  'CYAN',
  'YELLOW',
  'WHITE',
  'YELLOW',
  'CYAN',
  'GREEN',
  'PURPLE',
  'RED',
  'BLUE'],
 ['BLUE', 'GREEN', 'LTBLUE', 'LTGREEN', 'CYAN'],
 ['BROWN', 'ORANGE', 'YELLOW', 'ORANGE', 'BROWN'],
 ['BLACK', 'BLACK', 'BLACK', 'RED', 'BLACK', 'BLACK', 'YELLOW'],
 ['WHITE',
  'GRAY3',
  'LTGREEN',
  'GRAY2',
  'BLACK',
  'BLACK',
  'BLACK',
  'BLACK',
  'BLACK',
  'BLACK',
  'BLACK',
  'BLACK',
  'BLACK'],
 ['BLUE', 'LTBLUE', 'GRAY1', 'RED', 'GREEN']]

In [101]:
import os

from collections import defaultdict

for phase in range(1,16):
    sprite = '09'
    img = Image.new( 'RGBA', (width,height), "white")
    pixels = img.load()

    y = 0
    xpos_array, _ = xpos_history[190]
    xpos_array_head = xpos_array[:24]
    crop_left = min(xpos_array_head)
    crop_right = max(xpos_array_head)

    for i, x in enumerate(xpos_array_head):
        colors = [color_seq[0][i%len(color_seq[0])],
                  'WHITE',
                  color_seq[2][i%len(color_seq[2])]]
        origin = (x,y)
        paintSprite(origin, pixels, sprite, colors)

        colors = [color_seq[1][i%len(color_seq[1])],
                  'WHITE',
                  color_seq[5][i%len(color_seq[5])]]
        x2 = xpos_array[i+phase]
        origin = (x2,y)
        paintSprite(origin, pixels, sprite, colors)

        y += 8

    img = img.crop((crop_left - 10, 0, 
                    crop_right + 30, img.height))
    img.save(f'dna/phase/phase{phase}.png')


In [113]:
import os

from collections import defaultdict

for frame, (xpos_array, phase) in enumerate(xpos_history[:70]):
    sprite = '09'
    img = Image.new( 'RGBA', (width,height), "white")
    pixels = img.load()

    y = 0
    xpos_array_head = xpos_array[:24]
    crop_left = min(xpos_array_head)
    crop_right = max(xpos_array_head)

    for i, x in enumerate(xpos_array_head):
        colors = [color_seq[0][i%len(color_seq[0])],
                  'WHITE',
                  color_seq[2][i%len(color_seq[2])]]
        origin = (x,y)
        paintSprite(origin, pixels, sprite, colors)

        colors = [color_seq[1][i%len(color_seq[1])],
                  'WHITE',
                  color_seq[5][i%len(color_seq[5])]]
        x2 = xpos_array[i+phase]
        origin = (x2,y)
        paintSprite(origin, pixels, sprite, colors)

        y += 8

    img = img.crop((crop_left - 10, 0, 
                    crop_right + 30, img.height))
    img.save(f'dna/initial/frame{frame}.png')
    
    ofile = open(f'dna/initial/frame{frame}.tex','w')
    array_txt = ','.join([f"${hex(v)[2:].upper().zfill(2)}" for v in xpos_array])
    ofile.write(array_txt)
